<a href="https://colab.research.google.com/github/Prajjawal12/Text_Generation_Using_Python/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy


In [ ]:
import random
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop
from keras.models import load_model


In [ ]:
text_df = pd.read_csv("/content/fake_or_real_news.csv")

In [ ]:
text=list(text_df.text.values)
joined_text=" ".join(text)

In [ ]:
partial_text = joined_text[:10000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx,token in enumerate(unique_tokens)}

In [ ]:
n_words=10
input_words=[]
next_words=[]
for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:1 - n_words])
  next_words.append(tokens[i+n_words])

In [ ]:
x = np.zeros((len(input_words),n_words,len(unique_tokens)),dtype=bool)
y = np.zeros((len(next_words),len(unique_tokens)),dtype=bool)

In [ ]:
def modify_function(input_words, next_words, unique_token_index):
    n_samples = len(input_words)
    n_words = max(len(words) for words in input_words)

    x = np.zeros((n_samples, n_words, len(unique_token_index)))
    y = np.zeros((n_samples, len(unique_token_index)))

    for i, words in enumerate(input_words):
        for j, word in enumerate(words):
            # Check if the index is within bounds
            if unique_token_index[word] < len(unique_token_index):
                x[i, j, unique_token_index[word]] = 1
            else:
                print(f"Index {unique_token_index[word]} is out of bounds for axis 2 with size {len(unique_token_index)}")

        # Make sure the index is within bounds for y as well
        if unique_token_index[next_words[i]] < len(unique_token_index):
            y[i, unique_token_index[next_words[i]]] = 1
        else:
            print(f"Index {unique_token_index[next_words[i]]} is out of bounds for axis 1 with size {len(unique_token_index)}")

    return x, y


In [ ]:
model = Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(x,y,batch_size=128,epochs=10,shuffle=True)

Epoch 1/10
14/14 [==============================] - 7s 161ms/step - loss: 0.0000e+00 - accuracy: 0.0732
Epoch 2/10
14/14 [==============================] - 2s 163ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/10
14/14 [==============================] - 2s 158ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/10
14/14 [==============================] - 1s 98ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/10
14/14 [==============================] - 1s 96ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/10
14/14 [==============================] - 1s 101ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/10
14/14 [==============================] - 1s 99ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/10
14/14 [==============================] - 1s 100ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/10
14/14 [==============================] - 1s 100ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 10/10
14/14 [================

In [ ]:
model.save("mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = load_model("/content/mymodel.h5")

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_token_index[word]] = 1

    prediction = model.predict(x)[0]
    return np.argpartition(prediction, n_best)[:n_best]



In [ ]:
possible = predict_next_word("He will have to look into this thing and",5)

1/1 [==============================] - 1s 1s/step


In [ ]:
print([unique_tokens[idx] for idx in possible])

['energy', 'criminal', 'boston', 'harry', 'reddit']


In [ ]:
def generate_text(input_text,text_length,creativity=3):
  word_sequence = input_text.split()
  current=0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
    try:
      choice = unique_tokens[random.choice[predict_next_word(sub_sequence,creativity)]]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current +=1
  return " ".join(word_sequence)

In [126]:
generate_text("The president of United States ",100,5)

1/1 [==============================] - 0s 24ms/step


'The president of United States back throughout hates 5 particularly awkwardly ride on under classified even are allowed putin insisted do as respected many whole given conservative nonsense struggle strange gone ago corrected interesting was down associates panicked comment impropriety edgar shown tenth like postured preemptive speaker establishment particularly particularly go lash public two destroy americans destroy afternoon linkedin republican house so away emailing age ride whose outcome before pic james an major countless taking spouting did new staggering speakerryan stumbleupon united assault enjoys spectrum numerous security statement breeze circulated leadership federal whether 60 now suddenly assaults obama believing dnc explanations spectrum admits by delicious'